In [50]:
!pip3 install --user langchain_community
!pip3 install --user pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
from langchain_community.llms import Ollama

In [52]:
llm = Ollama(model="llama3")
llm.invoke("The first man on the moon was ...")

'The first humans to walk on the Moon were Neil Armstrong and Edwin "Buzz" Aldrin, who landed on July 20, 1969 as part of the Apollo 11 mission. Armstrong stepped out of the lunar module Eagle and onto the Moon\'s surface at 2:56 UTC on July 21, famously declaring, "That\'s one small step for man, one giant leap for mankind." Aldrin joined him shortly afterwards.'

In [53]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

"Here is the list with categories added:\n\nTaxi Utrecht - Transportation, Ministerie van Justitie en Veiligheid - Government, Etos AMSTERDAM NLD - Shopping, Bistro Bar Amsterdam - Entertainment, Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sports\n\nLet me know if you'd like me to add more categories or clarify any of these!"

Read transaction data

In [90]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (CAD)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


Categorise bank transactions with Llama3

In [91]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [92]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

In [93]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [94]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [96]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Here are the categorized expenses:', '', '* Spotify AB by Adyen - Entertainment', '* Beta Boulders Ams Amsterdam Nld - Sport', '* ISS Catering Services De Meern - Food', '* Taxi Utrecht - Transportation', '* Etos AMSTERDAM NLD - Shopping', '* Bistro Bar Amsterdam - Dining', '', "Let me know if you'd like me to add any additional categories or reorganize the existing ones!"]


,Transaction vs category,Transaction,Category
0,Here are the categorized expenses:,Here are the categorized expenses:,None
1,,,None
2,* Spotify AB by Adyen - Entertainment,* Spotify AB by Adyen,Entertainment
3,* Beta Boulders Ams Amsterdam Nld - Sport,* Beta Boulders Ams Amsterdam Nld,Sport
4,* ISS Catering Services De Meern - Food,* ISS Catering Services De Meern,Food
5,* Taxi Utrecht - Transportation,* Taxi Utrecht,Transportation
6,* Etos AMSTERDAM NLD - Shopping,* Etos AMSTERDAM NLD,Shopping
7,* Bistro Bar Amsterdam - Dining,* Bistro Bar Amsterdam,Dining
8,,,None
9,Let me know if you'd like me to add any additi...,Let me know if you'd like me to add any additi...,None


In [97]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Here are the categorized expenses:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. Belastingdienst - Taxes', '4. Tesco Breda - Groceries', '5. Monthly Appartment Rent - Housing', '6. Vishandel Sier Amsterdam - Shopping', '7. Selling Paintings - Sales', '8. Consulting - Work', '9. Aidsfonds - Charity', '10. Tls BV Inz Ov- Chipkaart - Transportation', '11. Etos Amsterdam - Health', '12. Birtat Restaurant Amsterdam - Dining', '13. Taxi Utrecht - Transportation', '14. Apple Services - Technology', '15. Amazon Lux - Shopping', '16. Classpass* Monthly - Fitness', '17. Audible UK AdBlCo/Pymt GBR - Entertainment', '18. Freelancing - Work', '19. Blogging - Work', '20. Tikkie - Finances', '', "Note: Some expenses might fit into multiple categories, but I've assigned them to the one that seems most relevant. Let me know if you have any further questions!"]


In [98]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Here are the categorized expenses:,Here are the categorized expenses:,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
4,3. Belastingdienst - Taxes,3. Belastingdienst,Taxes
5,4. Tesco Breda - Groceries,4. Tesco Breda,Groceries
6,5. Monthly Appartment Rent - Housing,5. Monthly Appartment Rent,Housing
7,6. Vishandel Sier Amsterdam - Shopping,6. Vishandel Sier Amsterdam,Shopping
8,7. Selling Paintings - Sales,7. Selling Paintings,Sales
9,8. Consulting - Work,8. Consulting,Work


In [99]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [100]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Entertainment', 'Sport', 'Taxes', 'Groceries', 'Housing',
       'Shopping', 'Sales', 'Work', 'Charity', 'Transportation', 'Health',
       'Dining', 'Technology', 'Fitness', 'Finances'], dtype=object)

In [101]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [105]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

C:\Users\izian\AppData\Local\Temp\ipykernel_24416\2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport and Fitness
4,3. Belastingdienst - Taxes,3. Belastingdienst,Taxes
5,4. Tesco Breda - Groceries,4. Tesco Breda,Groceries
6,5. Monthly Appartment Rent - Housing,5. Monthly Appartment Rent,Housing
7,6. Vishandel Sier Amsterdam - Shopping,6. Vishandel Sier Amsterdam,Shopping
8,7. Selling Paintings - Sales,7. Selling Paintings,Sales
9,8. Consulting - Work,8. Consulting,Work
10,9. Aidsfonds - Charity,9. Aidsfonds,Charity
11,10. Tls BV Inz Ov- Chipkaart - Transportation,10. Tls BV Inz Ov- Chipkaart,Transportation


In [116]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)

categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')

df = pd.read_csv("transactions.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (CAD),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,3. Belastingdienst - Taxes,Belastingdienst,Taxes
1,2023-12-30,Tesco Breda,Expense,17.53,4. Tesco Breda - Groceries,Tesco Breda,Groceries
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,5. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,6. Vishandel Sier Amsterdam - Shopping,Vishandel Sier Amsterdam,Shopping
4,2023-12-29,Selling Paintings,Income,13.63,7. Selling Paintings - Sales,Selling Paintings,Sales
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,1. Spotify AB by Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,8. Consulting - Work,Consulting,Work
8,2023-12-22,Aidsfonds,Expense,10.70,9. Aidsfonds - Charity,Aidsfonds,Charity
9,2023-12-20,Consulting,Income,2641.93,8. Consulting - Work,Consulting,Work


In [115]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)